In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from model import CNN_with_mask

from get_training_data import get_rally_result, get_padding_data, permute_feature

In [2]:
np.random.seed(44)
tf.random.set_seed(44)

In [3]:
df = pd.read_csv('new_test.csv')

In [4]:
# def replaced_by_mode(df, ex_feature):
#     if(ex_feature == 'Original'):
#         return df
#     col = ['Team', 'No.', 'Space', 'Action']
#     col.remove(ex_feature)
#     for c in col:
#         mode = df.mode(axis=0)[c][0]
#         replace = {list(df.groupby(c).groups.keys())[i]: mode for i in range(len(df.groupby(c)))}
#         df[c] = df[c].replace(replace)

#     return df

Feature: Team, No., Space, Action

In [5]:
feature_list = [['Player', 'Space'], ['Player', 'Action'], ['Player', 'Sequence'], ['Space', 'Action'], ['Space', 'Sequence'], ['Action', 'Sequence']]

In [6]:
for experiment in feature_list:
    ############# permutation #############################
    for f in experiment:
        df = permute_feature(df, f)
    ############# train for result ########################
    model = training(df)
    


   Game  Rally  Player Team Space Action  Errors  Nothing  Score
0    21   25.0   ITA_6  USA     6   JS_0     NaN      1.0    NaN
1    21   25.0  ITA_15  BRA     1    R_0     NaN      1.0    NaN
2    21   25.0   USA_2  BRA     6    G_0     NaN      1.0    NaN
3    21   25.0   ITA_6  BRA     2    A_0     NaN      NaN    1.0
4    21   25.0   ITA_5  USA     5    D_0     1.0      NaN    NaN
   Game  Rally  Player Team Space Action  Errors  Nothing  Score
0    21   25.0  ITA_15  USA     6    D_0     NaN      1.0    NaN
1    21   25.0  POL_19  BRA     1    A_0     NaN      1.0    NaN
2    21   25.0   ITA_6  BRA     6    R_0     NaN      1.0    NaN
3    21   25.0  FRA_19  BRA     2    G_1     NaN      NaN    1.0
4    21   25.0  FRA_19  USA     5    R_1     1.0      NaN    NaN
  Game  Rally  Player Team Space Action  Errors  Nothing  Score
0   14    6.0   ITA_7  POL     1   At_0     1.0      NaN    NaN
0   21   15.0   NED_7  BRA     Y   JS_0     NaN      1.0    NaN
1   21   15.0  FRA_14  BRA  

In [5]:
feature_1 = 'Space'
feature_2 = 'Player'
df = permute_feature(df, feature_1)
df = permute_feature(df, feature_2)

# df = replaced_by_mode(df, feature)

In [4]:
df.head(5)

,Game,Rally,Player,Team,Space,Action,Errors,Nothing,Score
0,21,25.0,USA_15,USA,Y,JS_0,NaN,1.0,NaN
1,21,25.0,BRA_18,BRA,7,R_0,NaN,1.0,NaN
2,21,25.0,BRA_1,BRA,2,G_0,NaN,1.0,NaN
3,21,25.0,BRA_23,BRA,2,A_0,NaN,NaN,1.0
4,21,25.0,USA_8,USA,8,D_0,1.0,NaN,NaN


In [5]:
def training(df):
    space_replace = {list(df.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df.groupby('Space')))}
    action_replace = {list(df.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df.groupby('Action')))}
    df = pd.get_dummies(df, columns=['Player'])
    df = df.replace(space_replace)
    df = df.replace(action_replace)
    # get label
    label, ignor = get_rally_result(df)
    # get training data
    rally_set, rally_space_set, rally_action_set= get_padding_data(df, ignor)
    rally_space_set = rally_space_set.squeeze()
    rally_action_set = rally_action_set.squeeze()
    rally_set_tensor = tf.convert_to_tensor(rally_set)
    rally_space_set_tensor = tf.convert_to_tensor(rally_space_set)
    rally_action_set_tensor = tf.convert_to_tensor(rally_action_set)
    rally_result_tensor = tf.convert_to_tensor(label)

    train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor)
    # setting
    rally_size = rally_set.shape[1]
    shot_size = 3
    feature_dim = (rally_set.shape[-1], len(df.groupby('Space'))+1, len(df.groupby('Action'))+1)
    space_embed_size = 8
    action_embed_size = 8
    shot_embed_size = 16
    # create model
    model = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
    optimizer = 'adam'
    loss = keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    epochs = 30
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True, monitor='val_loss')

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(train_x, train_y, epochs=epochs, validation_split=0.1, callbacks=[callbacks])

    return model

In [6]:
space_replace = {list(df.groupby('Space').groups.keys())[i]: i+1 for i in range(len(df.groupby('Space')))}
action_replace = {list(df.groupby('Action').groups.keys())[i]: i+1 for i in range(len(df.groupby('Action')))}

# df = pd.get_dummies(df, columns=['Team', 'No.'])
df = pd.get_dummies(df, columns=['Player'])
df = df.replace(space_replace)
df = df.replace(action_replace)

In [7]:
label, ignor = get_rally_result(df)

In [8]:
rally_set, rally_space_set, rally_action_set= get_padding_data(df, ignor)

# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(rally_space_set.shape)
print(rally_action_set.shape)
print(label.shape)

(1291, 19, 3, 127)
(1291, 19, 3, 1)
(1291, 19, 3, 1)
(1291, 2)


In [9]:
rally_space_set = rally_space_set.squeeze()
rally_action_set = rally_action_set.squeeze()

In [10]:
rally_set_tensor = tf.convert_to_tensor(rally_set)
rally_space_set_tensor = tf.convert_to_tensor(rally_space_set)
rally_action_set_tensor = tf.convert_to_tensor(rally_action_set)
rally_result_tensor = tf.convert_to_tensor(label)

In [14]:
from keras_self_attention import SeqWeightedAttention

In [11]:
class TestAttention(keras.layers.Layer):
    def __init__(self, input_dim=32):
        super().__init__()
        self.w = self.add_weight(
            shape=(input_dim, input_dim), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(input_dim,), initializer="random_normal", trainable=True)

    def call(self, inputs):
        # Hadamard product
        product_x = tf.matmul(inputs, self.w) + self.b
        soft_x = tf.nn.softmax(product_x)
        output = tf.multiply(inputs, soft_x)
        return output
    def compute_mask(self, inputs, mask=None):
        return mask
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'con_layer': self.con_layer
        })
        return config

In [12]:
# rally_num = rally_set.shape[0]
rally_size = rally_set.shape[1]
shot_size = 3
feature_dim = (rally_set.shape[-1], len(df.groupby('Space'))+1, len(df.groupby('Action'))+1)
space_embed_size = 8
action_embed_size = 8
shot_embed_size = 16

In [13]:
def create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size):
    '''
    framework: 
    1. 對 space, action 做 embeding, (input, output) = (feature_dim, embed_size)
    2. concat space, action, others 成一個 embedded vector for each atk, (input) =  ([feature_dim, embed_size, embed_size])
    3. 先做 embedding
    4. CNN, filters = shot_embed_size
    5. GRU
    '''
    # each input: 三個維度, rally shot feature
    input_others = keras.Input(shape=(rally_size, shot_size, feature_dim[0]))
    input_space = keras.Input(shape=(rally_size, shot_size))
    input_action = keras.Input(shape=(rally_size, shot_size))

    # space & action 先做 embedding, 再和 others concat
    embed_space_layer = Embedding(input_dim=feature_dim[1], output_dim=space_embed_size, mask_zero=True, name='Space_Embedding')
    embed_action_layer = Embedding(input_dim=feature_dim[2], output_dim=action_embed_size, mask_zero=True, name='Action_Embedding')
    masking_layer = Masking(mask_value=0)
    concat_layer = Concatenate(name='Input_Concat')

    attention_layer = TestAttention(input_dim=feature_dim[0]+space_embed_size+action_embed_size)

    embed_shot_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=3, name='Shot_Embedding')
    cnn_layer = CNN_with_mask(kernel_size=3, filters=shot_embed_size, strides=1, name='CNN_Layer')
    gru_layer = GRU(units=16, name='GRU_Layer')

    # gru_layer = GRU(return_sequences=True, units=16, name='GRU_Layer')
    # layer_attention = SeqWeightedAttention(return_attention=True)

    dense_layer = Dense(units=2, activation='softmax')

    # forward
    inputs = [input_others, input_space, input_action]

    embed_space = embed_space_layer(input_space)
    embed_action = embed_action_layer(input_action)
    masked_others = masking_layer(tf.cast(input_others, tf.float32))
    embed_input = concat_layer([masked_others, embed_space, embed_action])

    attention_input = attention_layer(embed_input)
    embed_shot = tf.squeeze(embed_shot_layer(attention_input), axis=2)

    # embed_shot = tf.squeeze(embed_shot_layer(embed_input), axis=2)

    cnn_output = cnn_layer(embed_shot)
    gru_output = gru_layer(cnn_output)

    # rally_represent, contributions = layer_attention(gru_output)
    # output = dense_layer(rally_represent)

    output = dense_layer(gru_output)
    model = keras.Model(inputs=inputs, outputs=output, name='Classification')

    # model_attention = tf.keras.Model(inputs=inputs, outputs=contributions, name='Attention')
    # return model, model_attention
    return model

In [14]:
# model, model_attention = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model = create_model(feature_dim, space_embed_size, action_embed_size, shot_embed_size)
model.summary()

Model: "Classification"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 19, 3, 127)] 0                                            
__________________________________________________________________________________________________
tf.cast (TFOpLambda)            (None, 19, 3, 127)   0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 19, 3)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 19, 3)]      0                                            
_____________________________________________________________________________________

In [15]:
def split_data(others_tensor, space_tensor, action_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.9)

    train_space = space_tensor[:split_persentage]
    train_action = action_tensor[:split_persentage]
    train_others = others_tensor[:split_persentage]
    train_label = label[:split_persentage]

    test_space = space_tensor[split_persentage:]
    test_action = action_tensor[split_persentage:]
    test_others = others_tensor[split_persentage:]
    test_label = label[split_persentage:]

    train_x = [train_others, train_space, train_action]
    train_y = train_label

    test_x = [test_others, test_space, test_action]
    test_y = test_label

    return train_x, train_y, test_x, test_y

In [16]:
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_space_set_tensor, rally_action_set_tensor, rally_result_tensor)

In [17]:
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True, monitor='val_loss')

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [19]:
weight_before = model.layers[8].get_weights()

In [20]:
model.fit(train_x, train_y, epochs=epochs, validation_split=0.1, callbacks=[callbacks])

Epoch 1/30
33/33 [==============================] - 3s 43ms/step - loss: 0.6564 - accuracy: 0.6513 - val_loss: 0.6039 - val_accuracy: 0.7094
Epoch 2/30
33/33 [==============================] - 1s 33ms/step - loss: 0.6428 - accuracy: 0.6600 - val_loss: 0.6074 - val_accuracy: 0.7094
Epoch 3/30
33/33 [==============================] - 1s 32ms/step - loss: 0.6433 - accuracy: 0.6600 - val_loss: 0.6079 - val_accuracy: 0.7094
Epoch 4/30
33/33 [==============================] - 1s 32ms/step - loss: 0.6434 - accuracy: 0.6600 - val_loss: 0.6153 - val_accuracy: 0.7094acy: 0.
Epoch 5/30
33/33 [==============================] - 1s 31ms/step - loss: 0.6424 - accuracy: 0.6600 - val_loss: 0.6085 - val_accuracy: 0.7094
Epoch 6/30
33/33 [==============================] - 1s 32ms/step - loss: 0.6418 - accuracy: 0.6600 - val_loss: 0.6083 - val_accuracy: 0.7094
Epoch 7/30
33/33 [==============================] - 1s 32ms/step - loss: 0.6416 - accuracy: 0.6600 - val_loss: 0.6112 - val_accuracy: 0.7094
Epoch 

In [21]:
weight_after = model.layers[8].get_weights()

In [22]:
weight_before

[array([[-0.08376861, -0.02498179,  0.01047854, ...,  0.02681371,
          0.02562301, -0.06401526],
        [-0.03941688,  0.04270202,  0.03665999, ..., -0.00363697,
         -0.05034636, -0.07477298],
        [ 0.0118912 , -0.02306752, -0.04930489, ..., -0.0185284 ,
         -0.01278042,  0.00308417],
        ...,
        [-0.06258168,  0.00010242, -0.07988643, ..., -0.03355335,
         -0.01033681,  0.03377532],
        [-0.0313025 ,  0.008078  ,  0.0051798 , ..., -0.07341696,
         -0.01812319,  0.04320578],
        [ 0.02175593,  0.02106545,  0.04403274, ...,  0.01319378,
         -0.09671459,  0.0850658 ]], dtype=float32),
 array([ 0.03038953, -0.00412433, -0.05329286,  0.05852119, -0.04122451,
        -0.04798507, -0.03668237,  0.04034596,  0.00456961,  0.06420854,
        -0.01609483, -0.04263012, -0.07461799,  0.02536712,  0.10310935,
         0.03627792,  0.02991605, -0.01822895, -0.02892245, -0.01732142,
         0.09463563, -0.00974691,  0.02731829,  0.08886766,  0.014

In [23]:
weight_after

[array([[-8.3777115e-02, -2.4981635e-02,  1.0478684e-02, ...,
          2.6812842e-02,  2.5626628e-02, -6.4017192e-02],
        [-3.9416879e-02,  4.2704612e-02,  3.6659990e-02, ...,
         -3.6369511e-03, -5.0346646e-02, -7.4772686e-02],
        [ 1.1890503e-02, -2.3068180e-02, -4.9203645e-02, ...,
         -1.8528562e-02, -1.2782124e-02,  3.0815790e-03],
        ...,
        [-6.2711060e-02, -2.2008362e-05, -8.0006540e-02, ...,
         -3.1670142e-02, -1.0778417e-02,  3.3656135e-02],
        [-3.1279765e-02,  8.1023788e-03,  5.2039437e-03, ...,
         -7.3896296e-02, -1.8020695e-02,  4.3219458e-02],
        [ 2.1656835e-02,  2.0967864e-02,  4.3943491e-02, ...,
          1.4978664e-02, -9.7223982e-02,  8.4818408e-02]], dtype=float32),
 array([ 2.62037441e-02, -8.23846459e-03, -5.72149456e-02,  5.42457141e-02,
        -4.52381074e-02, -5.19731306e-02, -4.07205746e-02,  3.61477248e-02,
         4.47482802e-04,  5.98933138e-02, -2.00890191e-02, -4.66553010e-02,
        -7.85864517e-0

In [25]:
tf.gradients(weight_before)

TypeError: gradients_v2() missing 1 required positional argument: 'xs'

In [21]:
# np.diag(model.weights[2])

In [22]:
# model.save_weights('bestModel/my_model_weights.h5')

In [23]:
result = model.evaluate(test_x, test_y)

5/5 [==============================] - 0s 6ms/step - loss: 0.6234 - accuracy: 0.6846


In [24]:
y_pred = model.predict(test_x)

In [25]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [26]:
argmax_y_pred = np.argmax(y_pred, axis=1)
argmax_test_y = np.argmax(test_y, axis=1)

In [27]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1

acc_score = accuracy_score(argmax_test_y, argmax_y_pred)
f1_score = f1(argmax_test_y, argmax_y_pred)
auc_score = roc_auc_score(test_y, y_pred)
BS = calculate_BS(test_y, y_pred, 2)

In [28]:
print('accuracy:    {:.2f}'.format(acc_score))
print('f1:          {:.2f}'.format(f1_score))
print('auc:         {:.2f}'.format(auc_score))
print('BS:          {:.2f}'.format(BS['BS'][0]))

accuracy:    0.68
f1:          0.00
auc:         0.52
BS:          0.22


In [35]:
def output_result(type, feature_list):
    if(type == 'permute'):
        output_path = 'permute_feature_result.txt'
        f = open(output_path, 'a')
        f.write('Permuted feature: ' + str(feature_list) + '\n')
        f.write('-'*40 + '\n')
        f.write('accuracy:    {:.2f}'.format(acc_score) + '\n')
        f.write('f1:          {:.2f}'.format(f1_score) + '\n')
        f.write('auc:         {:.2f}'.format(auc_score) + '\n')
        f.write('BS:          {:.2f}'.format(BS['BS'][0]) + '\n')
        f.write('\n' + '='*40 + '\n')
        f.close()
    elif(type == 'mode'):
        output_path = 'replaced_by_mode_result.txt'
        f = open(output_path, 'a')
        f.write('Fixed feature: ' + str(feature_list) + '\n')
        f.write('-'*40 + '\n')
        f.write('accuracy:    {:.2f}'.format(acc_score) + '\n')
        f.write('f1:          {:.2f}'.format(f1_score) + '\n')
        f.write('auc:         {:.2f}'.format(auc_score) + '\n')
        f.write('BS:          {:.2f}'.format(BS['BS'][0]) + '\n')
        f.write('\n' + '='*40 + '\n')
    elif(type == 'attention'):
        output_path = 'attention_result.txt'
        f = open(output_path, 'a')
        f.write('Permuted feature: ' + str(feature_list) + '\n')
        f.write('-'*40 + '\n')
        f.write('accuracy:    {:.2f}'.format(acc_score) + '\n')
        f.write('f1:          {:.2f}'.format(f1_score) + '\n')
        f.write('auc:         {:.2f}'.format(auc_score) + '\n')
        f.write('BS:          {:.2f}'.format(BS['BS'][0]) + '\n')
        f.write('\n' + '='*40 + '\n')
        f.close()

In [36]:
output_result('permute', [feature_1, feature_2])
# output_result('mode')